Необхідні для роботи імпорти

In [ ]:
import re, chardet, math
from collections import Counter
import pandas as pd

Змінні, які необхідні для роботи коду

In [2]:
file_path = r"C:\Users\blood\OneDrive\Рабочий стол\koran.txt"

Відкриваємо та форматуємо текст, щоб нам було зручно з ним працювати

In [ ]:
with open(file_path, "rb") as f:
    raw_data = f.read()
koduvannya = chardet.detect(raw_data)["encoding"]
print(f"Кодування: {koduvannya}")

def obrobka_text(filename, remove_spaces=False):
    with open(filename, "r", encoding=koduvannya, errors="replace") as file:
        text = file.read().lower()
    text = text.replace("ё", "е").replace("ъ", "ь")
    text = re.sub(r'[^а-я ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    if remove_spaces:
        text = text.replace(" ", "")
    return text

koran_chistenkiy = obrobka_text(file_path)

Кодування: utf-8


In [ ]:
def chastoty_bukv(tekst):
    N = len(tekst)
    counts = Counter(tekst)
    freq = {sym: count / N for sym, count in counts.items()}
    return freq, counts, N

def chastoty_bigram(tekst):
    bigrams = [tekst[i:i+2] for i in range(len(tekst) - 1)]
    N = len(bigrams)
    counts = Counter(bigrams)
    freq = {bg: count / N for bg, count in counts.items()}
    return freq, counts, N

def entropiya(freq_dict, n=1):
    """
    прийматиме freq_dict - словник {символ: ймовірність}
    n - довжина грам (1 для букв, 2 для біграм)
    """
    return -sum(p * math.log2(p) for p in freq_dict.values() if p > 0) / n